# Resolving Conflicts Using *Precedence Declarations*

This file shows how *shift/reduce* and *reduce/reduce* conflicts can be resolved using *operator precedence declarations*.
The following grammar is *ambiguous* because it does not specify the precedence of the arithmetical operators:
```
    expr : expr '+' expr
         | expr '-' expr
         | expr '*' expr
         | expr '/' expr
         | expr '^' expr
         | '(' expr ')'
         | NUMBER      
         ;
```

Instead of restructuring the grammar (stratification), we use Lezer's `@precedence` block to explicitly define:
1.  **Priority:** Multiplication (`times`) binds tighter than addition (`plus`).
2.  **Associativity:** Exponentiation `^` is right-associative (`@right`), others are left-associative (`@left`).

## Imports

In [ ]:
import { buildParser } from '@lezer/generator';
import { TreeCursor } from '@lezer/common';
import { ast2dot } from './AST2Dot';
import { cst2ast } from './CST2AST';
import { display } from "tslab";
import { instance } from "@viz-js/viz";
const viz = await instance();

## Grammar Definition

We define precedence levels and assign them to productions using the `!tag` syntax.
This tells Lezer: 
 * When reducing this rule, use the priority of `tag`

In [ ]:
const grammarPrecedence = `
    @precedence {
        power @right,
        times @left,
        plus  @left
    }

    @top Program { Expr }

    @tokens {
        Number { "0" | $[1-9] $[0-9]* }
        
        // Definiere die Operatoren als explizite Strings, 
        // damit sie im CST auch so heißen (z.B. node.name === "+")
        "+" "-" "*" "/" "^"
        
        space { $[ \t\n\r]+ }
        "(" ")"
    }

    Expr {
        Expr !plus  "+" Expr |
        Expr !plus  "-" Expr |
        Expr !times "*" Expr |
        Expr !times "/" Expr |
        Expr !power "^" Expr |
        "(" Expr ")"         |
        Number
    }
    
        @skip { space }
`;

const parser = buildParser(grammarPrecedence);
console.log("SUCCESS: Parser generated using Precedence Declarations.");

## Testing & Visualization

We use our generic `cst2ast` transformer.
Note: We define the `operators` list explicitly so the transformer knows which tokens should become root nodes (Pivot Promotion).

In [ ]:
const myOperators = ["+", "-", "*", "/", "^"];
const myListVars : string[] = []; // No lists here really, but required by signature

function test(input: string) {
    console.log(`\nInput: ${input}`);
    try {
        const tree = parser.parse(input);
        const ast = cst2ast(tree.cursor(), input, myOperators, myListVars);            
        const dot = ast2dot(ast);
        display.html(viz.renderString(dot, { format: "svg" }));
        
    } catch (e) {
        console.error(e);
    }
}

In [ ]:
test('2^3^4*5+6-7/8^9')

In [ ]:
test('1+2*3^4')

In [ ]:
test('1 * 2 + (3^4)^5')